In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense
from tensorflow.keras.optimizers import Adam,SGD
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras import initializers
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.layers import Dense, Flatten, Conv2D, AveragePooling2D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.constraints import max_norm


In [2]:
my_gpu = tf.config.experimental.list_physical_devices('GPU')
print(my_gpu)
tf.config.experimental.set_memory_growth(my_gpu[0],True)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
df = pd.read_csv('final_data.csv')

df.drop(['href', 'href','stress_address','city','state','county','img_href',"zip"], axis=1, inplace=True)
dfX = df.drop("price",axis=1)
dfX

,beds,bath,area,year_built,lot_size,walk_score,transit_score,bike_score,cooling,heating,has_pool,crime_rate_per_100000,minimum_wage,college_completion,unemployment_rate,median_household_income
0,3,2.0,1229,1983,0,68,39,63,0,0,0,442.965072,7.25,0.315,7.7,61807.0
1,3,1.0,1348,1913,0,70,0,46,0,0,0,102.396264,7.25,0.413,8.3,93587.0
2,3,1.0,1434,1941,0,45,19,39,1,2,0,177.018370,7.25,0.437,6.5,68975.0
3,3,2.0,1356,1961,0,11,15,33,2,2,0,721.521934,7.25,0.290,9.3,44773.0
4,5,1.5,2266,1940,0,5,0,42,0,3,0,18.351570,8.70,0.097,3.9,62596.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36330,2,1.0,928,1962,0,47,0,35,0,3,0,89.855758,11.25,0.194,8.2,52543.0
36331,3,1.5,1454,1956,0,42,0,44,0,0,0,290.721821,9.65,0.249,12.0,65051.0
36332,4,2.5,1772,2006,0,1,24,35,1,2,0,738.481892,9.00,0.344,8.1,56148.0
36333,3,1.5,1706,1930,0,55,0,46,0,3,0,521.234940,8.70,0.325,10.4,52503.0


In [4]:
X,y = np.array(dfX),np.array(df["price"])
scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [5]:
model = Sequential([
    Dense(units=128,input_shape=(len(X[0]),),kernel_initializer=initializers.RandomNormal(stddev=0.01),activation = LeakyReLU(alpha=0.01)),
    Dense(units=256,kernel_initializer=initializers.RandomNormal(stddev=0.01),activation = LeakyReLU(alpha=0.01)),
    Dense(units=256,kernel_initializer=initializers.RandomNormal(stddev=0.01),activation = LeakyReLU(alpha=0.01)),
    Dense(units=256,kernel_initializer=initializers.RandomNormal(stddev=0.01),activation = LeakyReLU(alpha=0.01)),
    Dense(units=1,kernel_initializer=initializers.RandomNormal(stddev=0.01),activation =  LeakyReLU(alpha=0.01))
])

In [6]:
def coeff_determination(y_true, y_pred):
    from tensorflow.keras import backend as K
    SS_res =  K.sum(K.square( y_true-y_pred ))
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) )
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [7]:
convergeAlarm = EarlyStopping(monitor='loss', min_delta=0, patience=50)
def lr_scheduler(epoch, lr):
    if epoch % 15 == 0 and epoch:
        return lr /1.15
    return lr

In [8]:
model.compile(optimizer= SGD(learning_rate=0.0001, momentum=0, nesterov=False), loss='mean_absolute_error', metrics = [coeff_determination])
history = model.fit(x=X_train, y=y_train, batch_size=128, epochs=40000, verbose=2,validation_split=0.2,shuffle=True, callbacks=[convergeAlarm,LearningRateScheduler(lr_scheduler, verbose=1)])

Epoch 1/40000

Epoch 00001: LearningRateScheduler reducing learning rate to 9.999999747378752e-05.
182/182 - 1s - loss: 630770.8125 - coeff_determination: -5.7383e-01 - val_loss: 646208.1250 - val_coeff_determination: -5.9180e-01
Epoch 2/40000

Epoch 00002: LearningRateScheduler reducing learning rate to 9.999999747378752e-05.
182/182 - 0s - loss: 630770.8750 - coeff_determination: -5.6275e-01 - val_loss: 646208.1250 - val_coeff_determination: -5.9180e-01
Epoch 3/40000

Epoch 00003: LearningRateScheduler reducing learning rate to 9.999999747378752e-05.
182/182 - 0s - loss: 630770.8750 - coeff_determination: -5.4899e-01 - val_loss: 646208.1250 - val_coeff_determination: -5.9180e-01
Epoch 4/40000

Epoch 00004: LearningRateScheduler reducing learning rate to 9.999999747378752e-05.
182/182 - 0s - loss: 630770.6875 - coeff_determination: -5.4865e-01 - val_loss: 646208.1250 - val_coeff_determination: -5.9180e-01
Epoch 5/40000

Epoch 00005: LearningRateScheduler reducing learning rate to 9.99

Epoch 37/40000

Epoch 00037: LearningRateScheduler reducing learning rate to 7.561436359537765e-05.
182/182 - 0s - loss: 630770.0625 - coeff_determination: -5.2669e-01 - val_loss: 646207.5625 - val_coeff_determination: -5.9180e-01
Epoch 38/40000

Epoch 00038: LearningRateScheduler reducing learning rate to 7.561436359537765e-05.
182/182 - 0s - loss: 630770.4375 - coeff_determination: -5.6026e-01 - val_loss: 646207.5625 - val_coeff_determination: -5.9180e-01
Epoch 39/40000

Epoch 00039: LearningRateScheduler reducing learning rate to 7.561436359537765e-05.
182/182 - 0s - loss: 630769.9375 - coeff_determination: -5.5727e-01 - val_loss: 646207.5625 - val_coeff_determination: -5.9180e-01
Epoch 40/40000

Epoch 00040: LearningRateScheduler reducing learning rate to 7.561436359537765e-05.
182/182 - 0s - loss: 630769.9375 - coeff_determination: -5.5893e-01 - val_loss: 646207.5625 - val_coeff_determination: -5.9180e-01
Epoch 41/40000

Epoch 00041: LearningRateScheduler reducing learning rate to

Epoch 73/40000

Epoch 00073: LearningRateScheduler reducing learning rate to 5.717532258131541e-05.
182/182 - 0s - loss: 630769.6875 - coeff_determination: -5.3300e-01 - val_loss: 646207.1875 - val_coeff_determination: -5.9180e-01
Epoch 74/40000

Epoch 00074: LearningRateScheduler reducing learning rate to 5.717532258131541e-05.
182/182 - 0s - loss: 630769.8750 - coeff_determination: -5.3321e-01 - val_loss: 646207.1875 - val_coeff_determination: -5.9180e-01
Epoch 75/40000

Epoch 00075: LearningRateScheduler reducing learning rate to 5.717532258131541e-05.
182/182 - 0s - loss: 630769.9375 - coeff_determination: -5.7016e-01 - val_loss: 646207.1875 - val_coeff_determination: -5.9180e-01
Epoch 76/40000

Epoch 00076: LearningRateScheduler reducing learning rate to 4.9717671809839494e-05.
182/182 - 0s - loss: 630769.6250 - coeff_determination: -5.3313e-01 - val_loss: 646207.1875 - val_coeff_determination: -5.9180e-01
Epoch 77/40000

Epoch 00077: LearningRateScheduler reducing learning rate t

Epoch 109/40000

Epoch 00109: LearningRateScheduler reducing learning rate to 3.759370156330988e-05.
182/182 - 0s - loss: 630769.4375 - coeff_determination: -5.8162e-01 - val_loss: 646206.8125 - val_coeff_determination: -5.9180e-01
Epoch 110/40000

Epoch 00110: LearningRateScheduler reducing learning rate to 3.759370156330988e-05.
182/182 - 0s - loss: 630769.5625 - coeff_determination: -5.7931e-01 - val_loss: 646206.8125 - val_coeff_determination: -5.9180e-01
Epoch 111/40000

Epoch 00111: LearningRateScheduler reducing learning rate to 3.759370156330988e-05.
182/182 - 0s - loss: 630769.4375 - coeff_determination: -5.6836e-01 - val_loss: 646206.8125 - val_coeff_determination: -5.9180e-01
Epoch 112/40000

Epoch 00112: LearningRateScheduler reducing learning rate to 3.759370156330988e-05.
182/182 - 0s - loss: 630769.1875 - coeff_determination: -5.6459e-01 - val_loss: 646206.8125 - val_coeff_determination: -5.9180e-01
Epoch 113/40000

Epoch 00113: LearningRateScheduler reducing learning ra

182/182 - 0s - loss: 630769.3125 - coeff_determination: -5.3222e-01 - val_loss: 646206.6250 - val_coeff_determination: -5.9180e-01
Epoch 145/40000

Epoch 00145: LearningRateScheduler reducing learning rate to 2.8426238714018837e-05.
182/182 - 0s - loss: 630769.3750 - coeff_determination: -5.3699e-01 - val_loss: 646206.6250 - val_coeff_determination: -5.9180e-01
Epoch 146/40000

Epoch 00146: LearningRateScheduler reducing learning rate to 2.8426238714018837e-05.
182/182 - 0s - loss: 630768.7500 - coeff_determination: -5.6497e-01 - val_loss: 646206.6250 - val_coeff_determination: -5.9180e-01
Epoch 147/40000

Epoch 00147: LearningRateScheduler reducing learning rate to 2.8426238714018837e-05.
182/182 - 0s - loss: 630769.1875 - coeff_determination: -5.4690e-01 - val_loss: 646206.6250 - val_coeff_determination: -5.9180e-01
Epoch 148/40000

Epoch 00148: LearningRateScheduler reducing learning rate to 2.8426238714018837e-05.
182/182 - 0s - loss: 630769.1250 - coeff_determination: -5.3491e-01 

Epoch 180/40000

Epoch 00180: LearningRateScheduler reducing learning rate to 2.1494321117643267e-05.
182/182 - 0s - loss: 630768.8750 - coeff_determination: -5.7674e-01 - val_loss: 646206.3750 - val_coeff_determination: -5.9180e-01
Epoch 181/40000

Epoch 00181: LearningRateScheduler reducing learning rate to 1.869071401534197e-05.
182/182 - 0s - loss: 630769.0000 - coeff_determination: -5.4625e-01 - val_loss: 646206.3750 - val_coeff_determination: -5.9180e-01
Epoch 182/40000

Epoch 00182: LearningRateScheduler reducing learning rate to 1.8690714568947442e-05.
182/182 - 0s - loss: 630768.8750 - coeff_determination: -5.4318e-01 - val_loss: 646206.3750 - val_coeff_determination: -5.9180e-01
Epoch 183/40000

Epoch 00183: LearningRateScheduler reducing learning rate to 1.8690714568947442e-05.
182/182 - 0s - loss: 630769.1875 - coeff_determination: -5.4144e-01 - val_loss: 646206.3750 - val_coeff_determination: -5.9180e-01
Epoch 184/40000

Epoch 00184: LearningRateScheduler reducing learning

182/182 - 0s - loss: 630768.9375 - coeff_determination: -5.5947e-01 - val_loss: 646206.1875 - val_coeff_determination: -5.9180e-01
Epoch 216/40000

Epoch 00216: LearningRateScheduler reducing learning rate to 1.4132865544524975e-05.
182/182 - 0s - loss: 630768.7500 - coeff_determination: -5.5854e-01 - val_loss: 646206.1875 - val_coeff_determination: -5.9180e-01
Epoch 217/40000

Epoch 00217: LearningRateScheduler reducing learning rate to 1.4132865544524975e-05.
182/182 - 0s - loss: 630768.7500 - coeff_determination: -5.2053e-01 - val_loss: 646206.1875 - val_coeff_determination: -5.9180e-01
Epoch 218/40000

Epoch 00218: LearningRateScheduler reducing learning rate to 1.4132865544524975e-05.
182/182 - 0s - loss: 630769.0000 - coeff_determination: -5.5462e-01 - val_loss: 646206.1875 - val_coeff_determination: -5.9180e-01
Epoch 219/40000

Epoch 00219: LearningRateScheduler reducing learning rate to 1.4132865544524975e-05.
182/182 - 0s - loss: 630768.7500 - coeff_determination: -5.2327e-01 